## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [38]:
TRAIN_PATH = 'C:/Users/Professional/Documents/GitHub/clf_Maximkin_2020/data/news_train.txt'
TEST_PATH = 'C:/Users/Professional/Documents/GitHub/clf_Maximkin_2020/data/news_test.txt'

In [39]:
lines = list(open(TRAIN_PATH, 'r', encoding='utf-8'))
lines[0]

'sport\tОвечкин пожертвовал детской хоккейной школе автомобиль\tНападающий «Вашингтон Кэпиталз» Александр Овечкин передал детской хоккейной школе автомобиль, полученный им после окончания Матча всех звезд Национальной хоккейной лиги (НХЛ). Об этом сообщается на официальном сайте лиги.Автомобиль Honda Accord был подарен хоккеисту по решению спонсоров мероприятия. Игрок НХЛ пожертвовал машину спортивной школе Nova Cool Cats Special Hockey Inc., которая расположена в штате Вирджиния.Овечкин общается с 10-летней девочкой Анной Шоб с синдромом Дауна, которая занимается в этой школе и является поклонницей спортсмена. В сентябре форвард пообедал вместе с юной хоккеисткой в японском ресторане.Матч всех звезд НХЛ в Коламбусе (штат Огайо) завершился победой команды «Джонатана Тэйвза» над командой «Ника Фолиньо» со счетом 17:12. Овечкин выступал за проигравший коллектив. Россиянин отметился тремя результативными передачами.\n'

In [40]:
import random
random.shuffle(lines)

In [41]:
from collections import Counter

Counter([line.split('\t')[0] for line in lines[:15000]])

Counter({'economics': 2080,
         'business': 554,
         'science': 2156,
         'life': 2033,
         'culture': 2053,
         'sport': 2215,
         'style': 284,
         'media': 2111,
         'forces': 1225,
         'travel': 289})

In [7]:
'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

In [42]:
import pymorphy2
import numpy as np
import torch
import re
import pandas as pd
import gensim

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)

In [43]:
def tokenize_text(text, regex_string = r'\b\w+\b'):
    return re.findall(regex_string, text.lower())

def tokenize_dataset(lines, regex_string = r'\b\w+\b'):
    result = []
    for line in lines:
        label, headline, text = line.split('\t')
        headline = tokenize_text(headline, regex_string)
        text = tokenize_text(text, regex_string)
        result.append(dict())
        result[-1]['label'] = label
        result[-1]['headline'] = headline
        result[-1]['text'] = text
    return result

def lemmatize_dataset(lines):
    morph_analyzer = pymorphy2.MorphAnalyzer()
    for line in lines:
        line['headline'] = list(map(lambda x: morph_analyzer.parse(x)[0].normal_form, line['headline']))
        line['text'] = list(map(lambda x: morph_analyzer.parse(x)[0].normal_form, line['text']))

In [44]:
train_data = tokenize_dataset(lines)
lemmatize_dataset(train_data)

In [45]:
mapping_dict = {'economics': 0,
                'business': 1,
                'science': 2,
                'life': 3,
                'culture': 4,
                'sport': 5,
                'style': 6,
                'media': 7,
                'forces': 8,
                'travel': 9
               }

In [46]:
def map_labels(labels, mapping_dict):
    return labels.map(lambda label: mapping_dict[label]).to_list()

2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

In [47]:
train_df = pd.DataFrame(train_data)

In [48]:
word2vec = gensim.models.Word2Vec(train_df['text'].append(train_df['headline']))

In [49]:
print(word2vec.wv.most_similar(positive=['россия']),'\n',
      word2vec.wv.most_similar(positive=['сша']), '\n', 
      word2vec.wv.most_similar(positive=['windows']))

[('украина', 0.711722731590271), ('рф', 0.7109173536300659), ('страна', 0.6423488855361938), ('белоруссия', 0.6368809938430786), ('крым', 0.635071873664856), ('европа', 0.6152746677398682), ('греция', 0.6043877601623535), ('турция', 0.5781927108764648), ('российский', 0.5667814016342163), ('республика', 0.565057635307312)] 
 [('великобритания', 0.7096855044364929), ('китай', 0.6480938792228699), ('франция', 0.6311972737312317), ('индия', 0.6072044372558594), ('япония', 0.5947538614273071), ('германия', 0.5647455453872681), ('австралия', 0.5645737051963806), ('американский', 0.5614012479782104), ('страна', 0.5591923594474792), ('израиль', 0.5215529203414917)] 
 [('ios', 0.9275631904602051), ('android', 0.8822500705718994), ('firefox', 0.8690578937530518), ('iphone', 0.8636046648025513), ('phone', 0.8586814403533936), ('планшет', 0.845851719379425), ('ipad', 0.8457365036010742), ('galaxy', 0.8391407132148743), ('nexus', 0.8320471048355103), ('mini', 0.823755145072937)]


3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [51]:
def weighted_average_vectorize(X, word2vec):
    word2vec_dict = dict(zip(word2vec.wv.index2word, word2vec.wv.vectors))
    dim = len(next(iter(word2vec_dict.values())))        
    
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(X)
    max_idf = max(tfidf.idf_)
    word2weight = defaultdict(
            lambda: max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
    )
    
    return np.array(
        [np.mean([word2vec[w] * word2weight[w] for w in words if w in word2vec] or [np.zeros(dim)], axis=0) for words in X])

In [52]:
x_train = weighted_average_vectorize(train_df['text'], word2vec)
y_train = map_labels(train_df['label'], mapping_dict)

<ipython-input-51-fb9fd793412f>:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  [np.mean([word2vec[w] * word2weight[w] for w in words if w in word2vec] or [np.zeros(dim)], axis=0) for words in X])
<ipython-input-51-fb9fd793412f>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  [np.mean([word2vec[w] * word2weight[w] for w in words if w in word2vec] or [np.zeros(dim)], axis=0) for words in X])


In [53]:
test_lines = list(open(TEST_PATH, 'r', encoding='utf-8'))
test_data = tokenize_dataset(test_lines)
lemmatize_dataset(test_data)
test_df = pd.DataFrame(test_data)

In [54]:
x_test = weighted_average_vectorize(test_df['text'], word2vec)
y_test = map_labels(test_df['label'], mapping_dict)

<ipython-input-51-fb9fd793412f>:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  [np.mean([word2vec[w] * word2weight[w] for w in words if w in word2vec] or [np.zeros(dim)], axis=0) for words in X])
<ipython-input-51-fb9fd793412f>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  [np.mean([word2vec[w] * word2weight[w] for w in words if w in word2vec] or [np.zeros(dim)], axis=0) for words in X])


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [56]:
clf = LogisticRegression(n_jobs = -1)
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
accuracy_score(y_test, predictions)

0.8433333333333334

In [57]:
clf = SVC()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
accuracy_score(y_test, predictions)

0.848